In [1]:
import pandas as pd 
import csv  

In [22]:
# Import csv file with total cases and deaths by US state and county
# Data was downloaded from https://www.kaggle.com/imdevskp/corona-virus-report#usa_county_wise.csv
file = "Data/usa_county_wise.csv"
kaggle_df = pd.read_csv(file)
kaggle_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/20,0,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/20,0,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/20,0,0


In [15]:
# Only keep rows for US states (i.e. remove Guam, Virgin Islands, etc.)
kaggle_us = kaggle_df.loc[kaggle_df["iso2"] == "US",:]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,0
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,0
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,0
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,0
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,0
10,84001011,US,USA,840,1011.0,Bullock,Alabama,US,32.100305,-85.712655,"Bullock, Alabama, US",1/22/20,0,0
11,84001013,US,USA,840,1013.0,Butler,Alabama,US,31.753001,-86.680575,"Butler, Alabama, US",1/22/20,0,0
12,84001015,US,USA,840,1015.0,Calhoun,Alabama,US,33.774837,-85.826304,"Calhoun, Alabama, US",1/22/20,0,0
13,84001017,US,USA,840,1017.0,Chambers,Alabama,US,32.913601,-85.390727,"Chambers, Alabama, US",1/22/20,0,0
14,84001019,US,USA,840,1019.0,Cherokee,Alabama,US,34.178060,-85.606390,"Cherokee, Alabama, US",1/22/20,0,0


In [16]:
# Remove columns not needed (UID, iso2(country/territory), iso3(country/territory), & code3)
kaggle_rem_cols = kaggle_us[["FIPS", "Admin2","Province_State", "Lat", "Long_", "Combined_Key", "Date", "Confirmed", "Deaths"]]

,FIPS,Admin2,Province_State,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
5,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,0
6,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,0
7,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,0
8,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,0
9,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,0


In [21]:
# Rename columns (Admin 2 to County, Province_State to State, Long_ to Lng)
kaggle_renamed = kaggle_rem_cols.rename(columns={"Admin2":"County", "Province_State":"State", "Long_":"Lng"})
kaggle_renamed.head(40)

,FIPS,County,State,Lat,Lng,Combined_Key,Date,Confirmed,Deaths
5,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,0
6,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,0
7,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,0
8,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,0
9,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,0
10,1011.0,Bullock,Alabama,32.100305,-85.712655,"Bullock, Alabama, US",1/22/20,0,0
11,1013.0,Butler,Alabama,31.753001,-86.680575,"Butler, Alabama, US",1/22/20,0,0
12,1015.0,Calhoun,Alabama,33.774837,-85.826304,"Calhoun, Alabama, US",1/22/20,0,0
13,1017.0,Chambers,Alabama,32.913601,-85.390727,"Chambers, Alabama, US",1/22/20,0,0
14,1019.0,Cherokee,Alabama,34.178060,-85.606390,"Cherokee, Alabama, US",1/22/20,0,0
